** This file is the initial code with the data analysis and formatting

In [205]:
import requests
from dotenv import load_dotenv
from datetime import datetime
import json
import os
import pandas as pd
import pprint
import zipfile
import io
import pandas as pd
import requests
import pytz

static_files_url = "https://oct-gtfs-emasagcnfmcgeham.z01.azurefd.net/public-access/GTFSExport.zip"
base_url = "https://nextrip-public-api.azure-api.net/octranspo/"
endpoint = "gtfs-rt-tp/beta/v1/TripUpdates?format=json"
trip_updates_api_url = f"{base_url}{endpoint}"
load_dotenv()
oc_transpo_api_key = os.getenv('api_key')

load_dotenv()
header = {
        "Ocp-Apim-Subscription-Key": os.getenv('api_key')
    }

In [206]:
def time_string_to_seconds(time_str):
    """
    Converts a GTFS time string (HH:MM:SS) to total seconds from midnight.
    Handles times beyond 24:00:00 as per GTFS spec.
    Returns pd.NA for missing/invalid input.
    """
    if pd.isna(time_str):
        return pd.NA
    try:
        parts = str(time_str).split(':')
        if len(parts) == 3:
            h, m, s = map(int, parts)
            return h * 3600 + m * 60 + s
        elif len(parts) == 2: # Handle HH:MM format if it appears (GTFS usually has HH:MM:SS)
            h, m = map(int, parts)
            return h * 3600 + m * 60
        else:
            return pd.NA
    except ValueError:
        return pd.NA

In [207]:
def load_static_files_to_df(url):
    gtfs_dataframes = {}
    try:
        response = requests.get(url, stream = True)
        response.raise_for_status()

        print("Opening zip file in memory")
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            filenames = z.namelist()
            print("Files in the zip folder are: ", filenames)

            for file in filenames:
                if file.endswith('.txt'):
                    try:
                        with z.open(file) as f:
                            df = pd.read_csv(f)
                            gtfs_dataframes[file.replace('.txt', '')] = df
                            print(f"Successfully loaded {file} with {len(df)} rows.")
                    except pd.errors.EmptyDataError:
                        print("Warning: file is empty and could not be loaded")
                    except Exception as e:
                        print("Error loading file")
                else:
                    print(f"skiping non .txt file {file}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the zip file: {e}")
    except zipfile.BadZipFile as e:
        print(f"Error: The downloaded file is not a valid zip file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return gtfs_dataframes

In [208]:
def fetch_realtime_trip_updates(api_url, api_key, timezone_obj):
    parsed_updates = []

    if not api_key:
        print("Error: OC_TRANSPO_API_KEY is not set. Cannot fetch real-time data.")
        return pd.DataFrame()

    print(f"\n--- Fetching Real-Time Trip Updates from API_URL:")
    try:
        response = requests.get(api_url, headers=header)
        response.raise_for_status()
        trip_updates_data = response.json()
        fetch_time_utc = datetime.now(pytz.utc)

        if trip_updates_data:
            for entity in trip_updates_data.get('Entity', []):
                if "TripUpdate" in entity and entity["TripUpdate"] is not None:
                    trip_update = entity["TripUpdate"]
                    
                    trip_info = trip_update.get("Trip", {})
                    trip_id = str(trip_info.get("TripId")) if trip_info.get("TripId") is not None else None
                    route_id = str(trip_info.get("RouteId")) if trip_info.get("RouteId") is not None else None
                    direction_id = int(trip_info.get("DirectionId")) if trip_info.get("DirectionId") is not None else None
                    vehicle_id = None
                    vehicle_info = trip_update.get("Vehicle")
                    if vehicle_info:
                        vehicle_id = vehicle_info.get("Id")
                    for stop_time_update in trip_update.get("StopTimeUpdate", []):
                        stop_sequence_no = int(stop_time_update.get("StopSequence")) if stop_time_update.get("StopSequence") is not None else None
                        stop_id = str(stop_time_update.get("StopId")) if stop_time_update.get("StopId") is not None else None
                        schedule_relationship = int(stop_time_update.get("ScheduleRelationship")) if stop_time_update.get("ScheduleRelationship") is not None else None
                        arrival_data = stop_time_update.get("Arrival")
                        if arrival_data is None:
                            arrival_data = {}
                        arrival_timestamp = arrival_data.get("Time")
                        departure_data = stop_time_update.get("Departure")
                        if departure_data is None:
                            departure_data = {}
                        departure_timestamp = departure_data.get("Time")
                        parsed_updates.append({
                            'fetch_time_utc': fetch_time_utc,
                            'trip_id': trip_id,
                            'route_id': route_id,
                            'direction_id': direction_id,
                            'vehicle_id': vehicle_id,
                            'stop_id': stop_id,
                            'stop_sequence_no': stop_sequence_no,
                            'schedule_relationship': schedule_relationship,
                            'arrival_timestamp_rt': arrival_timestamp,
                            'departure_timestamp_rt': departure_timestamp
                        })
        else:
            print("No 'Entity' data found in the TripUpdates response.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching real-time trip updates: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from API response: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during real-time data processing: {e}")

    if parsed_updates:
        trip_updates_df = pd.DataFrame(parsed_updates)

        # Copying 'departure_timestamp_rt' to 'arrival_timestamp_rt' 
        # since the 'arrival_timestamp_rt' is always null where ever the stop_sequence_no is 1
        condition = (trip_updates_df['arrival_timestamp_rt'].isnull()) & (trip_updates_df['stop_id'] == 1)
        
        trip_updates_df.loc[condition, 'arrival_timestamp_rt'] = trip_updates_df.loc[condition, 'departure_timestamp_rt']
        print(f"Copied departure_timestamp_rt to arrival_timestamp_rt for {condition.sum()} rows where arrival was null and stop_id was '1'.")

        
        # The time we are getting from the Real-time data are already in UTC as per GTFS-RT specification
        trip_updates_df['arrival_datetime_utc'] = pd.to_datetime(trip_updates_df['arrival_timestamp_rt'], unit='s', errors='coerce', utc=True)
        trip_updates_df['departure_datetime_utc'] = pd.to_datetime(trip_updates_df['departure_timestamp_rt'], unit='s', errors='coerce', utc=True)

        # Additionally I am converting that UTC to EDT, since the time values from the static files are already in EDT
        trip_updates_df['arrival_datetime_local'] = trip_updates_df['arrival_datetime_utc'].dt.tz_convert(timezone_obj)
        trip_updates_df['departure_datetime_local'] = trip_updates_df['departure_datetime_utc'].dt.tz_convert(timezone_obj)

        trip_updates_df = trip_updates_df.drop(columns=['arrival_timestamp_rt', 'departure_timestamp_rt'], errors='ignore')

        initial_rows = len(trip_updates_df)
        trip_updates_df = trip_updates_df.dropna(subset=['trip_id', 'stop_id', 'stop_sequence_no']).copy()
        if len(trip_updates_df) < initial_rows:
            print(f"Removed {initial_rows - len(trip_updates_df)} real-time updates due to missing essential IDs.")

        print(f"Successfully parsed {len(trip_updates_df)} real-time trip updates.")
        print("\n--- Real-Time Trip Updates DataFrame (Head) ---")
        print(trip_updates_df.head())
        print("\n--- Real-Time Trip Updates DataFrame Dtypes ---")
        print(trip_updates_df.dtypes)
        return trip_updates_df
    else:
        print("No real-time trip updates were parsed.")
        return pd.DataFrame()

In [209]:
def standardize_static_gtfs_dtypes(static_gtfs_df):
    if 'routes' in static_gtfs_df:
        df = static_gtfs_df['routes']
        df['route_id'] = df['route_id'].astype(str)
        
    if 'stops' in static_gtfs_df:
        df = static_gtfs_df['stops']
        df['stop_id'] = df['stop_id'].astype(str)
        df['stop_name'] = df['stop_name'].astype(str)

    if 'trips' in static_gtfs_df:
        df = static_gtfs_df['trips']
        df['trip_id'] = df['trip_id'].astype(str)
        df['route_id'] = df['route_id'].astype(str)
        df['service_id'] = df['service_id'].astype(str)
        if 'direction_id' in df.columns:
            df['direction_id'] = pd.to_numeric(df['direction_id'], errors='coerce').astype(int)

    if 'stop_times' in static_gtfs_df:
        df = static_gtfs_df['stop_times']
        df['trip_id'] = df['trip_id'].astype(str)
        df['stop_id'] = df['stop_id'].astype(str)
        df['stop_sequence'] = pd.to_numeric(df['stop_sequence'], errors='coerce').astype(int)

        df['scheduled_arrival_seconds'] = df['arrival_time'].apply(time_string_to_seconds).astype('Int64')
        df['scheduled_departure_seconds'] = df['departure_time'].apply(time_string_to_seconds).astype('Int64')

        df = df.drop(columns=['arrival_time', 'departure_time'], errors='ignore')

        # Removed 'pickup_type', 'drop_off_type', 'timepoint' conversion as per user's request
        # for col in ['pickup_type', 'drop_off_type', 'timepoint']:
        #     if col in df.columns:
        #         df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

    print("\n--- GTFS Static DataFrames after Type Standardization ---")
    for name, df in static_gtfs_df.items():
        print(f"\nDataFrame for '{name}' Dtypes:")
        print(df.dtypes)
    return static_gtfs_df

In [210]:
def standardize_merged_scheduled_datetimes(merged_df, current_timezone):
    print("\n--- Standardizing Scheduled Datetimes in Merged DataFrame ---")

    if 'arrival_datetime_local' not in merged_df.columns or not pd.api.types.is_datetime64_any_dtype(merged_df['arrival_datetime_local']):
        print("Warning: 'arrival_datetime_utc' not found or not datetime. Cannot standardize scheduled times.")
        return merged_df

    # Extracting the date from the real-time arrival data
    merged_df['service_date_local'] = merged_df['arrival_datetime_local'].dt.normalize()

    merged_df['scheduled_arrival_timedelta'] = pd.to_timedelta(merged_df['scheduled_arrival_seconds'], unit='s', errors='coerce')
    merged_df['scheduled_departure_timedelta'] = pd.to_timedelta(merged_df['scheduled_departure_seconds'], unit='s', errors='coerce')

    # Combining the local date from the real-time event with the scheduled_arrival_timedelta to form a naive local datetime
    merged_df['scheduled_arrival_datetime_local'] = merged_df['service_date_local'] + merged_df['scheduled_arrival_timedelta']
    merged_df['scheduled_departure_datetime_local'] = merged_df['service_date_local'] + merged_df['scheduled_departure_timedelta']

    merged_df = merged_df.drop(columns=[
        'scheduled_arrival_seconds', 'scheduled_departure_seconds',
        'scheduled_arrival_timedelta', 'scheduled_departure_timedelta',
        'service_date_local'
    ], errors='ignore')

    print("\n--- Merged DataFrame Dtypes after Scheduled Datetime Standardization ---")
    print(merged_df[[
        'arrival_datetime_local', 'scheduled_arrival_datetime_local',
        'departure_datetime_local', 'scheduled_departure_datetime_local'
    ]].dtypes)

    return merged_df

In [211]:
if __name__ == "__main__":
    print("--- Starting OC Transpo Prediction Accuracy Analysis ---")

    # Step 1: Load Static GTFS Data
    gtfs_static_data = load_static_files_to_df(static_files_url)
    if not gtfs_static_data:
        print("Exiting due to failure to load static GTFS data.")
        exit()

    # Fetching the local timezone from Agency.txt or agency dataframe
    try:
        agency_timezone_str = gtfs_static_data['agency']['agency_timezone'].iloc[0]
        fetched_timezone = pytz.timezone(agency_timezone_str)
        print(f"Fetched timezone from agency.txt: {agency_timezone_str}")
    except KeyError:
        print("Error: 'agency_timezone' column not found in agency.txt or 'agency' table not found.")
        print("Defaulting to 'America/Toronto'.")
        fetched_timezone = pytz.timezone('America/Toronto')
    except IndexError:
        print("Error: No agency found in agency.txt.")
        print("Defaulting to 'America/Toronto'.")
        fetched_timezone = pytz.timezone('America/Toronto')
    except pytz.UnknownTimeZoneError:
        print(f"Error: Unknown timezone '{agency_timezone_str}' fetched from agency.txt.")
        print("Defaulting to 'America/Toronto'.")
        fetched_timezone = pytz.timezone('America/Toronto')

    # Step 2: Standardize Static GTFS Data Types
    gtfs_static_data = standardize_static_gtfs_dtypes(gtfs_static_data)

    # Step 3: Fetch Real-time Trip Updates (includes internal datetime and ID type standardization)
    trip_updates_df = fetch_realtime_trip_updates(trip_updates_api_url, oc_transpo_api_key, fetched_timezone)
    if trip_updates_df.empty:
        print("Exiting due to failure to load real-time trip updates.")
        exit()

    # Step 4: Merge Real-Time Data with Static Scheduled Data
    print("\n--- Merging Real-Time Data with Static Scheduled Data ---")

    required_static_dfs_for_merge = ['stop_times', 'trips', 'routes', 'stops']
    if not all(df_name in gtfs_static_data for df_name in required_static_dfs_for_merge):
        print(f"FATAL ERROR: Missing one or more essential static GTFS DataFrames for merging ({required_static_dfs_for_merge}). Exiting.")
        exit()

    merged_df = pd.merge(
        trip_updates_df,
        gtfs_static_data['stop_times'],
        left_on=['trip_id', 'stop_id', 'stop_sequence_no'],
        right_on=['trip_id', 'stop_id', 'stop_sequence'],
        how='left',
        suffixes=('_rt', '_scheduled_static_time_data')
    )

    merged_df = pd.merge(
        merged_df,
        gtfs_static_data['trips'][['trip_id', 'route_id', 'service_id', 'trip_short_name', 'trip_headsign', 'direction_id']],
        on='trip_id',
        how='left',
        suffixes=('_merged', '_trip_static_info')
    )
    merged_df.rename(columns={'direction_id': 'direction_id_static'}, inplace=True)

    merged_df = pd.merge(
        merged_df,
        gtfs_static_data['routes'][['route_id', 'route_short_name', 'route_long_name', 'route_type']],
        left_on='route_id_merged',
        right_on='route_id',
        how='left',
        suffixes=('_route_rt', '_route_static_info')
    )

    merged_df = pd.merge(
        merged_df,
        gtfs_static_data['stops'][['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],
        on='stop_id',
        how='left'
    )

--- Starting OC Transpo Prediction Accuracy Analysis ---
Opening zip file in memory
Files in the zip folder are:  ['agency.txt', 'stops.txt', 'routes.txt', 'trips.txt', 'stop_times.txt', 'calendar.txt', 'calendar_dates.txt', 'shapes.txt', 'feed_info.txt']
Successfully loaded agency.txt with 1 rows.
Successfully loaded stops.txt with 5566 rows.
Successfully loaded routes.txt with 125 rows.
Successfully loaded trips.txt with 79811 rows.


C:\Users\Admin\AppData\Local\Temp\ipykernel_50240\1990940059.py:16: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\Admin\AppData\Local\Temp\ipykernel_50240\1990940059.py:16: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


Successfully loaded stop_times.txt with 3172835 rows.
Successfully loaded calendar.txt with 20 rows.
Successfully loaded calendar_dates.txt with 50 rows.
Successfully loaded shapes.txt with 689200 rows.
Successfully loaded feed_info.txt with 1 rows.
Fetched timezone from agency.txt: Canada/Eastern

--- GTFS Static DataFrames after Type Standardization ---

DataFrame for 'agency' Dtypes:
agency_id            int64
agency_name         object
agency_url          object
agency_timezone     object
agency_lang         object
agency_phone        object
agency_fare_url     object
agency_email       float64
dtype: object

DataFrame for 'stops' Dtypes:
stop_id                 object
stop_code               object
stop_name               object
tts_stop_name          float64
stop_desc              float64
stop_lat               float64
stop_lon               float64
zone_id                float64
stop_url               float64
location_type          float64
parent_station          object
stop_tim

In [212]:
    merged_df.iloc[:, :12]

,fetch_time_utc,trip_id,route_id_merged,direction_id_merged,vehicle_id,stop_id,stop_sequence_no,schedule_relationship,arrival_datetime_utc,departure_datetime_utc,arrival_datetime_local,departure_datetime_local
0,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,1203,7,0,2025-07-29 02:53:38+00:00,NaT,2025-07-28 22:53:38-04:00,NaT
1,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,1204,8,0,2025-07-29 02:54:37+00:00,NaT,2025-07-28 22:54:37-04:00,NaT
2,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,1205,9,0,2025-07-29 02:55:27+00:00,NaT,2025-07-28 22:55:27-04:00,NaT
3,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,1206,10,0,2025-07-29 02:56:12+00:00,NaT,2025-07-28 22:56:12-04:00,NaT
4,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,9590,11,0,2025-07-29 02:56:47+00:00,NaT,2025-07-28 22:56:47-04:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
5667,2025-07-29 02:53:28.363669+00:00,31385020,82,0,4803,88,52,0,2025-07-29 03:17:24+00:00,NaT,2025-07-28 23:17:24-04:00,NaT
5668,2025-07-29 02:53:28.363669+00:00,31385020,82,0,4803,89,53,0,2025-07-29 03:17:41+00:00,NaT,2025-07-28 23:17:41-04:00,NaT
5669,2025-07-29 02:53:28.363669+00:00,31385020,82,0,4803,90,54,0,2025-07-29 03:17:59+00:00,NaT,2025-07-28 23:17:59-04:00,NaT
5670,2025-07-29 02:53:28.363669+00:00,31385020,82,0,4803,91,55,0,2025-07-29 03:18:16+00:00,NaT,2025-07-28 23:18:16-04:00,NaT


In [213]:
    final_analysis_df = standardize_merged_scheduled_datetimes(merged_df, fetched_timezone)

    # --- Post-Merge Filtering and Initial Summary ---
    initial_merged_rows = len(final_analysis_df)
    final_analysis_df['has_scheduled_data'] = final_analysis_df['scheduled_arrival_datetime_local'].notna() | \
                                               final_analysis_df['scheduled_departure_datetime_local'].notna()

    final_analysis_df = final_analysis_df[final_analysis_df['has_scheduled_data']].copy()
    rows_without_scheduled = initial_merged_rows - len(final_analysis_df)
    if rows_without_scheduled > 0:
        print(f"\nRemoved {rows_without_scheduled} real-time updates that could not be matched to static scheduled stop_times.")

    print(f"\n--- Final DataFrame for Analysis (Total records: {len(final_analysis_df)}) ---")
    print(final_analysis_df[[
        'fetch_time_utc', 'trip_id', 'route_id', 'route_short_name', 'trip_headsign',
        'stop_id', 'stop_name', 'stop_sequence_no',
        'scheduled_arrival_datetime_local', 'arrival_datetime_local',
        'scheduled_departure_datetime_local', 'departure_datetime_local',
        'arrival_datetime_utc', 'arrival_datetime_utc',
        'schedule_relationship', 'vehicle_id',
        'direction_id_merged', 'direction_id_trip_static_info'
    ]].head(10))

    print("\n--- Final DataFrame Dtypes for Key Columns ---")
    print(final_analysis_df[[
        'trip_id', 'route_id', 'route_short_name', 'trip_headsign',
        'stop_id', 'stop_name', 'stop_sequence_no',
        'scheduled_arrival_datetime_local', 'arrival_datetime_local',
        'scheduled_departure_datetime_local', 'departure_datetime_local',
        'arrival_datetime_utc', 'arrival_datetime_utc',
        'schedule_relationship', 'vehicle_id',
        'direction_id_merged', 'direction_id_trip_static_info'
    ]].dtypes)

    print("\n--- Project Setup and Data Preparation Complete ---")
    print("You now have 'final_analysis_df' ready for delay calculations and accuracy analysis.")


--- Standardizing Scheduled Datetimes in Merged DataFrame ---

--- Merged DataFrame Dtypes after Scheduled Datetime Standardization ---
arrival_datetime_local                datetime64[ns, Canada/Eastern]
scheduled_arrival_datetime_local      datetime64[ns, Canada/Eastern]
departure_datetime_local              datetime64[ns, Canada/Eastern]
scheduled_departure_datetime_local    datetime64[ns, Canada/Eastern]
dtype: object

Removed 179 real-time updates that could not be matched to static scheduled stop_times.

--- Final DataFrame for Analysis (Total records: 5493) ---
                    fetch_time_utc   trip_id route_id route_short_name  \
0 2025-07-29 02:53:28.363669+00:00  12310020       12               12   
1 2025-07-29 02:53:28.363669+00:00  12310020       12               12   
2 2025-07-29 02:53:28.363669+00:00  12310020       12               12   
3 2025-07-29 02:53:28.363669+00:00  12310020       12               12   
4 2025-07-29 02:53:28.363669+00:00  12310020       12 

In [214]:
#final_analysis_df['trip_id', 'route_id', 'route_short_name', 'trip_headsign, stop_id', 'stop_name', 'stop_sequence_no', 'arrival_datetime_local', 'departure_datetime_local', 'scheduled_arrival_datetime_local', 'scheduled_departure_datetime_local', 'schedule_relationship', 'vehicle_id', 'direction_id_merged', 'direction_id_trip_static_info']
columns_to_print = [
    'trip_id',
    'route_id',
    'route_short_name',
    'trip_headsign',
    'stop_id',
    'stop_name',
    'stop_sequence_no',
    'arrival_datetime_local',
    'departure_datetime_local',
    'scheduled_arrival_datetime_local',
    'scheduled_departure_datetime_local',
    'arrival_datetime_utc',
    'schedule_relationship',
    'vehicle_id',
    'direction_id_merged',
    'direction_id_trip_static_info'
]

# Select and print the specified columns
final_analysis_df[columns_to_print]

,trip_id,route_id,route_short_name,trip_headsign,stop_id,stop_name,stop_sequence_no,arrival_datetime_local,departure_datetime_local,scheduled_arrival_datetime_local,scheduled_departure_datetime_local,arrival_datetime_utc,schedule_relationship,vehicle_id,direction_id_merged,direction_id_trip_static_info
0,12310020,12,12,Parliament ~ Parlement,1203,EARL ARMSTRONG ARENA,7,2025-07-28 22:53:38-04:00,NaT,2025-07-28 22:45:20-04:00,2025-07-28 22:45:20-04:00,2025-07-29 02:53:38+00:00,0,6632,0,1.0
1,12310020,12,12,Parliament ~ Parlement,1204,JASMINE / OGILVIE,8,2025-07-28 22:54:37-04:00,NaT,2025-07-28 22:46:19-04:00,2025-07-28 22:46:19-04:00,2025-07-29 02:54:37+00:00,0,6632,0,1.0
2,12310020,12,12,Parliament ~ Parlement,1205,JASMINE / ARROWSMITH,9,2025-07-28 22:55:27-04:00,NaT,2025-07-28 22:47:09-04:00,2025-07-28 22:47:09-04:00,2025-07-29 02:55:27+00:00,0,6632,0,1.0
3,12310020,12,12,Parliament ~ Parlement,1206,JASMINE / BEAVERHILL,10,2025-07-28 22:56:12-04:00,NaT,2025-07-28 22:47:54-04:00,2025-07-28 22:47:54-04:00,2025-07-29 02:56:12+00:00,0,6632,0,1.0
4,12310020,12,12,Parliament ~ Parlement,9590,JASMINE / OGILVIE,11,2025-07-28 22:56:47-04:00,NaT,2025-07-28 22:48:29-04:00,2025-07-28 22:48:29-04:00,2025-07-29 02:56:47+00:00,0,6632,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5667,31385020,82,82,Lincoln Fields,88,CARLING / DUNDEE,52,2025-07-28 23:17:24-04:00,NaT,2025-07-28 23:15:43-04:00,2025-07-28 23:15:43-04:00,2025-07-29 03:17:24+00:00,0,4803,0,1.0
5668,31385020,82,82,Lincoln Fields,89,CARLING / ALPINE,53,2025-07-28 23:17:41-04:00,NaT,2025-07-28 23:16:00-04:00,2025-07-28 23:16:00-04:00,2025-07-29 03:17:41+00:00,0,4803,0,1.0
5669,31385020,82,82,Lincoln Fields,90,CARLING / TAVISTOCK,54,2025-07-28 23:17:59-04:00,NaT,2025-07-28 23:16:18-04:00,2025-07-28 23:16:18-04:00,2025-07-29 03:17:59+00:00,0,4803,0,1.0
5670,31385020,82,82,Lincoln Fields,91,CARLING / CONNAUGHT,55,2025-07-28 23:18:16-04:00,NaT,2025-07-28 23:16:35-04:00,2025-07-28 23:16:35-04:00,2025-07-29 03:18:16+00:00,0,4803,0,1.0


In [218]:
final_analysis_df[final_analysis_df['stop_sequence_no'] == 1][columns_to_print]

,trip_id,route_id,route_short_name,trip_headsign,stop_id,stop_name,stop_sequence_no,arrival_datetime_local,departure_datetime_local,scheduled_arrival_datetime_local,scheduled_departure_datetime_local,arrival_datetime_utc,schedule_relationship,vehicle_id,direction_id_merged,direction_id_trip_static_info


In [216]:
print("\n--- Calculating Delay in Seconds ---")
final_analysis_df['arrival_delay_sec'] = (
    final_analysis_df['arrival_datetime_local'] - final_analysis_df['scheduled_arrival_datetime_local']
).dt.total_seconds()

final_analysis_df['departure_delay_sec'] = (
    final_analysis_df['departure_datetime_local'] - final_analysis_df['scheduled_departure_datetime_local']
).dt.total_seconds()

final_analysis_df


--- Calculating Delay in Seconds ---


,fetch_time_utc,trip_id,route_id_merged,direction_id_merged,vehicle_id,stop_id,stop_sequence_no,schedule_relationship,arrival_datetime_utc,departure_datetime_utc,...,route_long_name,route_type,stop_name,stop_lat,stop_lon,scheduled_arrival_datetime_local,scheduled_departure_datetime_local,has_scheduled_data,arrival_delay_sec,departure_delay_sec
0,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,1203,7,0,2025-07-29 02:53:38+00:00,NaT,...,Blair <> Tunney's Pasture,3.0,EARL ARMSTRONG ARENA,45.438437,-75.601062,2025-07-28 22:45:20-04:00,2025-07-28 22:45:20-04:00,True,498.0,NaN
1,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,1204,8,0,2025-07-29 02:54:37+00:00,NaT,...,Blair <> Tunney's Pasture,3.0,JASMINE / OGILVIE,45.439692,-75.597763,2025-07-28 22:46:19-04:00,2025-07-28 22:46:19-04:00,True,498.0,NaN
2,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,1205,9,0,2025-07-29 02:55:27+00:00,NaT,...,Blair <> Tunney's Pasture,3.0,JASMINE / ARROWSMITH,45.439707,-75.594902,2025-07-28 22:47:09-04:00,2025-07-28 22:47:09-04:00,True,498.0,NaN
3,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,1206,10,0,2025-07-29 02:56:12+00:00,NaT,...,Blair <> Tunney's Pasture,3.0,JASMINE / BEAVERHILL,45.441472,-75.594655,2025-07-28 22:47:54-04:00,2025-07-28 22:47:54-04:00,True,498.0,NaN
4,2025-07-29 02:53:28.363669+00:00,12310020,12,0,6632,9590,11,0,2025-07-29 02:56:47+00:00,NaT,...,Blair <> Tunney's Pasture,3.0,JASMINE / OGILVIE,45.442860,-75.596306,2025-07-28 22:48:29-04:00,2025-07-28 22:48:29-04:00,True,498.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5667,2025-07-29 02:53:28.363669+00:00,31385020,82,0,4803,88,52,0,2025-07-29 03:17:24+00:00,NaT,...,Baseline <> Lincoln Fields & Tunney's Pasture,3.0,CARLING / DUNDEE,45.361172,-75.790835,2025-07-28 23:15:43-04:00,2025-07-28 23:15:43-04:00,True,101.0,NaN
5668,2025-07-29 02:53:28.363669+00:00,31385020,82,0,4803,89,53,0,2025-07-29 03:17:41+00:00,NaT,...,Baseline <> Lincoln Fields & Tunney's Pasture,3.0,CARLING / ALPINE,45.362198,-75.788617,2025-07-28 23:16:00-04:00,2025-07-28 23:16:00-04:00,True,101.0,NaN
5669,2025-07-29 02:53:28.363669+00:00,31385020,82,0,4803,90,54,0,2025-07-29 03:17:59+00:00,NaT,...,Baseline <> Lincoln Fields & Tunney's Pasture,3.0,CARLING / TAVISTOCK,45.363112,-75.786480,2025-07-28 23:16:18-04:00,2025-07-28 23:16:18-04:00,True,101.0,NaN
5670,2025-07-29 02:53:28.363669+00:00,31385020,82,0,4803,91,55,0,2025-07-29 03:18:16+00:00,NaT,...,Baseline <> Lincoln Fields & Tunney's Pasture,3.0,CARLING / CONNAUGHT,45.363747,-75.785048,2025-07-28 23:16:35-04:00,2025-07-28 23:16:35-04:00,True,101.0,NaN


In [219]:
mean_arrival_delay = final_analysis_df['arrival_delay_sec'].mean()
mae_arrival = final_analysis_df['arrival_delay_sec'].abs().mean()
on_time_arrival_pct = (final_analysis_df['arrival_delay_sec'].abs() <= 60).mean() * 100

print(f"Mean Arrival Delay: {mean_arrival_delay:.2f} seconds")
print(f"MAE (Mean Absolute Error): {mae_arrival:.2f} seconds")
print(f"% of Arrivals within ±1 min: {on_time_arrival_pct:.2f}%")

Mean Arrival Delay: -330.60 seconds
MAE (Mean Absolute Error): 546.96 seconds
% of Arrivals within ±1 min: 57.66%
